In [7]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('databases/merged_gtfs.db')
gtfs_names = ['routes', 'stop_times', 'stops', 'transfers', 'trips']

for name in gtfs_names:
    df = pd.read_csv(f"gtfs/{name}.txt")
    df.to_sql(name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

/tmp/ipykernel_37501/2957384310.py:9: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"gtfs/{name}.txt")


In [9]:
import sqlite3

def test_query(db_path: str, query: str):
    """
    Test function to run a SQL query on the specified SQLite database.

    :param db_path: Path to the SQLite database
    :param query: SQL query to execute
    :return: Results of the query
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Close the connection
        conn.close()

        return results
    except sqlite3.Error as e:
        return {"error": f"SQLite error: {str(e)}"}

# Path to the merged GTFS database
db_path = "merged_gtfs.db"

# Example query to fetch all routes from the 'routes' table
query = "SELECT * FROM routes LIMIT 5;"  # You can modify this query as needed

# Run the query
results = test_query(db_path, query)

# Print the results
if isinstance(results, list):
    for row in results:
        print(row)
else:
    print(results)  # In case of an error, print the error message


('5-T01-y08-1', 1, 'T1', None, 0, 'F47922', '000000')
('11-MGr-n-y08-1', 1, 'MGrn', None, 1, '0AAE53', '000000')
('11-MRe-1-y08-1', 1, 'MRed1', None, 1, 'E91D2F', '000000')
('11-MRe-2-y08-1', 1, 'MRed2', None, 1, 'E91D2F', '000000')
('10', 1, '10', 'Gold Souq Bus Stn - Al Quoz Bus Stn', 3, '6F2E90', 'FFFFFF')


In [ ]:
import streamlit as st
from tools import GeocodingTool, GTFSCoordinatorTool, CurrentDateTime, TransitRoutingTool
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI

# Initialize tools
gtfs_tool = GTFSCoordinatorTool()
geocode_tool = GeocodingTool()
curr_dt_tool = CurrentDateTime()
route_tool = TransitRoutingTool()

# Initialize LLM with system message for proper query format
llm = ChatOpenAI(
                base_url="https://api.sambanova.ai/v1/",
                api_key=st.secrets["SAMBANOVA_API_KEY"], 
                streaming=True,
                model="Meta-Llama-3.1-405B-Instruct",
            )

# Define tools
tools = [
    Tool(
        name="GeocodingTool",
        func=geocode_tool.run,
        description="Fetches geocoding information from TomTom's API based on user inputs"
    ),
    # Tool(
    #     name="TransitRoutingTool",
    #     func=route_tool.run,
    #     description="Fetches route information from Bing Routes API based on user inputs",
    # ),
    Tool(
        name="CurrentDateTime",
        func=curr_dt_tool.run,
        description="Returns the current date and time."
    ),
    Tool(
        name="GTFSCoordinatorTool",
        func=gtfs_tool.run,
        description="Handles queries related to public transit data such as schedules, routes, and stop information"
    ),
]

sys_msg = """
You are an intelligent assistant specializing in public transit for Dubai. Your role is to assist users with queries related to transit schedules, routes, stops, geolocation, and route planning. Follow these instructions:

Understand the User Query:
Analyze the user's input to determine whether it requires:

1. Geolocation data (via GeocodingTool).
2. Transit schedule or stop information (via GTFSCoordinatorTool).
3. Time-sensitive data (via CurrentDateTime).
4. Route planning for public transit (via TransitRoutingTool).

Provide Accurate Responses:

1. Use the GTFSCoordinatorTool to query transit data from the GTFS database.
2. Use the GeocodingTool to resolve place names into precise geographic coordinates.
3. Use CurrentDateTime to determine or adjust for time-based responses.
4. Use the TransitRoutingTool to generate optimized public transit routes between waypoints based on user inputs like start location, destination, and preferences.

Output Format:
Ensure responses are clear and structured:

- Provide transit details like schedules, routes, or stops in an easy-to-read format.
- Return geolocation results with relevant coordinates and descriptions.
- Include time-based context (current or user-specified) when appropriate.
- For route planning, provide step-by-step directions, transit modes, and estimated travel times.
- Pass origin and destination separately.

- Common bus names (E101, E400, etc.) are found in the routes table -> route_short_name column.

If you don't recognize a particular station or stop name: 
- Common stop names (Al Ghubaiba Bus Stn, Gold Souq Bus Stn, etc.) are found in the stops table -> stop_headsign column

Stay Contextual:

- Respond in a concise, Dubai-specific context.
- Include relevant details about transit services like bus, metro, and tram lines in Dubai.
- For route planning, ensure recommendations prioritize public transit and avoid unnecessary detours.
- Include options or alternatives when possible (e.g., alternate routes or travel modes).

Collaborate with Tools:
Use the tools efficiently to deliver accurate and timely information. Avoid making assumptions about transit data, geolocation, or routing without querying the relevant tool.
"""

# Initialize agent with tools and system message
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, system_message=sys_msg
) # Test query
query = ""
response = agent.run(query)
print("Response: ", response)




> Entering new AgentExecutor chain...
Thought: To find out when the next E400 bus leaves from Union Bus Stop 8, I need to first get the current date and time to ensure I'm looking at the correct schedule, then use the GTFSCoordinatorTool to get the schedule information for the E400 bus at Union Bus Stop 8.

Action: CurrentDateTime
Action Input: None
Observation: The current date and time is: 2025-01-26 19:03:28
Thought:Thought: Now that I have the current date and time, I can use the GTFSCoordinatorTool to get the schedule information for the E400 bus at Union Bus Stop 8.

Action: GTFSCoordinatorTool
Action Input: {"route": "E400", "stop": "Union Bus Stop 8", "time": "2025-01-26 19:03:28"}SELECT T1.trip_id, T1.arrival_time, T1.departure_time 
FROM stop_times AS T1 
INNER JOIN stops AS T2 
ON T1.stop_id = T2.stop_id 
INNER JOIN trips AS T3 
ON T1.trip_id = T3.trip_id 
INNER JOIN routes AS T4 
ON T3.route_id = T4.route_id 
WHERE T2.stop_name = 'Union Bus Stop 8' 
AND T4.route_short_nam

In [91]:
import requests

response = requests.get(f"http://dev.virtualearth.net/REST/V1/Routes/Transit?o=json&wp.0=london&wp.1=leeds&avoid=minimizeTolls&key={st.secrets["BINGMAPS_KEY"]}")
response

<Response [200]>

In [79]:
response.json()['resourceSets'][0]['resources'][0]['travel']

dict_keys(['__type', 'bbox', 'id', 'currencyCode', 'distanceUnit', 'durationUnit', 'routeLegs', 'trafficCongestion', 'trafficDataUsed', 'travelDistance', 'travelDuration', 'travelDurationTraffic', 'travelMode'])

In [81]:
travelDistance = response.json()['resourceSets'][0]['resources'][0]['travelDistance']
travelDuration = response.json()['resourceSets'][0]['resources'][0]['travelDuration']

travelDistance, travelDuration

(276.646, 16188)

In [66]:
start_coords = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['actualStart']
end_coords = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['actualEnd'] 

start_coords, end_coords

({'type': 'Point', 'coordinates': [51.507434, -0.128006]},
 {'type': 'Point', 'coordinates': [53.800686, -1.55029]})

In [68]:
response.json()['resourceSets'][0]

{'estimatedTotal': 1,
 'resources': [{'__type': 'Route:http://schemas.microsoft.com/search/local/ws/rest/v1',
   'bbox': [51.507099, -1.55029, 53.800768, -0.128006],
   'id': 'v70,h-1666434989,i0,a2,cen-US,dAAAAAAAAAAA1,y1,s1,m4,o1,t0,wSN3OvvLASUDqB3WRQlnAvw2~AXf2AverdXoJAADgAQAAgD8A0~TG9uZG9uLCBHcmVhdGVyIExvbmRvbiwgVW5pdGVkIEtpbmdkb201~~~~~~~~v12,w72_QXn3mSkCHqMKf4c34vw2~AXf2AveTsIoHAADgAQAAAAAA0~TGVlZHMsIFdlc3QgWW9ya3NoaXJlLCBVbml0ZWQgS2luZ2RvbQ2~~~~~~~~v12,k1,pZ3NpLXBvaQ2-VHJ1ZQ2',
   'currencyCode': '',
   'distanceUnit': 'Kilometer',
   'durationUnit': 'Second',
   'routeLegs': [{'actualEnd': {'type': 'Point',
      'coordinates': [53.800686, -1.55029]},
     'actualStart': {'type': 'Point', 'coordinates': [51.507434, -0.128006]},
     'alternateVias': [],
     'endLocation': {'bbox': [53.698959, -1.800431, 53.952049, -1.165114],
      'name': 'Leeds, West Yorkshire, United Kingdom',
      'point': {'type': 'Point', 'coordinates': [53.800701, -1.550264]},
      'address': {'adminD

In [69]:
itinerary_items = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['itineraryItems']

instructions = [item['instruction']['text'] for item in itinerary_items] 
instructions

['Walk from London, Greater London, United Kingdom to Trafalgar Square',
 'Bus: 453 - Marylebone',
 'Walk from Baker Street Station to Baker Street Underground Station',
 'Subway: Jubilee - Wembley Park',
 'Walk from Finchley Road Underground Station to Finchley Road Station (Stop CL)',
 'Bus: M20',
 'Walk from Bus Stn (Stand 1) to Leeds, West Yorkshire, United Kingdom']

In [48]:
itinerary_items[0]

{'compassDirection': '',
 'details': [{'endPathIndices': [4],
   'maneuverType': 'Walk',
   'mode': 'Transit',
   'roadType': 'NotApplicable',
   'startPathIndices': [0]}],
 'exit': '',
 'iconType': 'Walk',
 'instruction': {'maneuverType': 'DepartStart',
  'text': 'Walk from London, Greater London, United Kingdom to Trafalgar Square'},
 'isRealTimeTransit': False,
 'maneuverPoint': {'type': 'Point', 'coordinates': [51.507434, -0.128006]},
 'realTimeTransitDelay': 0,
 'sideOfStreet': 'Unknown',
 'tollZone': '',
 'towardsRoadName': '',
 'transitTerminus': '',
 'travelDistance': 0.129,
 'travelDuration': 92,
 'travelMode': 'Transit'}